In [1]:
import pandas as pd
import os
import boto3
import numpy as  np
import pandas as pd
import json
import time
from multiprocessing.dummy import Pool
import shutil

In [2]:
os.chdir('//FS2\RTO\\CIS-PD MUSC\\')
client = boto3.client('athena', region_name = 'us-west-2')
user_id_pairings = pd.read_csv("decoded_forms\\videoID.csv")
subjenroll = pd.read_sas("subjenrollment.sas7bdat")
startstop = pd.read_csv("decoded_forms\\WatchStartStop.csv")

# set path to folder containing subject records
path = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
# set path to destination folder
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Record Correlation'
#---------------------------------------------------------------------------------------------------------

In [3]:
# Medication Reports
table_med = 'Table10.csv'
dest_ext_med = 'Medication Reports'
file_name_med = 'med_timepoints.csv'

# Symptom Reports
table_symt = 'Table11.csv'
dest_ext_symt = 'Symptom Reports'
file_name_symt = 'symt_timepoints.csv'

# Diaries
table_diar = 'Table12.csv'
dest_ext_diar = 'Diaries'
file_name_diar = 'diar_timepoints.csv'

# Load Consolidated DataFrames
timepoints_med = pd.read_csv(os.path.join(dest, dest_ext_med, file_name_med), parse_dates = [1])
timepoints_symt = pd.read_csv(os.path.join(dest, dest_ext_symt, file_name_symt), parse_dates = [1])
timepoints_diar = pd.read_csv(os.path.join(dest, dest_ext_diar, file_name_diar), parse_dates = [1])

In [76]:
# Generate Subdirectories to Append Time Interval Watch Data
# 'if statements' Required Because Record Type Paths Have Different Character Numbers
Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)

for file in Records['SaveFilePaths']:
    
    if file[:81] == '//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Medication Reports':
        # create directories by day
        try:
            os.mkdir(file[:97])
        except(FileExistsError):
            continue
        except(FileNotFoundError):
            # create directories by month
            try:
                os.mkdir(file[:94])
                os.mkdir(file[:97])
            # create directories by subject
            except(FileNotFoundError):
                os.mkdir(file[:86])
                os.mkdir(file[:94])
                os.mkdir(file[:97])

    if file[:78] == '//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Symptom Reports':
        # create directories by day
        try:
            os.mkdir(file[:94])
        except(FileExistsError):
            continue
        except(FileNotFoundError):
            # create directories by month
            try:
                os.mkdir(file[:91])
                os.mkdir(file[:94])
            # create directories by subject
            except(FileNotFoundError):
                os.mkdir(file[:83])
                os.mkdir(file[:91])
                os.mkdir(file[:94])
                    

    if file[:70] == '//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Diaries':
        # create directories by day
        try:
            os.mkdir(file[:86])
        except(FileExistsError):
            continue
        except(FileNotFoundError):
            # create directories by month
            try:
                os.mkdir(file[:83])
                os.mkdir(file[:86])
            # create directories by subject
            except(FileNotFoundError):
                    os.mkdir(file[:75])
                    os.mkdir(file[:83])
                    os.mkdir(file[:86])

In [4]:
paginator = client.get_paginator('get_query_results')

In [5]:
def user_id_4_to_6(user_id_4):
    
    # iterate through the rows of 'videoID.csv' file in 'decoded_forms' directory of 'CIS-PD MUSC' directory
    # file contains a column for both 4-digit and 6-digit subject ID
    for index, row in user_id_pairings.iterrows():
        if (int(row['SubjectCode']) == int(user_id_4)):
            
            # return 'None' if the SubjectCode (4-digit) has no corresponding 6-digit ID (Subj ID Athena cell in csv is empty)
            if np.isnan(row['Subj ID Athena']):
                return None
            
            # if a 6-digit code is contained within the row that has the 4-digit code, return it
            return int(row['Subj ID Athena'])
        
    # 'None' is returned if no row contains the 4-digit code
    return None

In [6]:
def generate_query(DateSubjRecords, S6, S):
    template1 = "SELECT user_id-" + str(int(S6) - int(S)) + " AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z\n"
    template2 = "FROM watch_accelerometer\nwhere cohort != 'test'\nand user_id = "
    template3 = "\nand ((timestamp >= timestamp '"
    template4 = "' and timestamp < timestamp '"
    template5 = "')"
    
    startTime = str(DateSubjRecords[0][0])[:23]
    endTime = str(DateSubjRecords[0][1])[:23]

    query = template1 + template2 + S6 + template3 + startTime + template4 + endTime + template5
    
    looptemplate1 = "\nor (timestamp >= timestamp '"
    looptemplate2 = "' and timestamp < timestamp '"
    looptemplate3 = "')"

    for i in range(1, len(DateSubjRecords)):
        query = (query + looptemplate1 + str(DateSubjRecords[i][0])[:23] + looptemplate2 + 
                 str(DateSubjRecords[i][1])[:23] + looptemplate3)
        
    query = query + ")\nORDER BY timestamp"
    
    # generate query from row in form509
    return query

In [7]:
def call_query(query):
    database = 'clinician_input_study'
    output_location = 's3://aws-athena-query-results-arunjayaraman-us-west-2/'
    encryption_option = 'SSE_S3'

    query_info = client.start_query_execution(
        QueryString = query, 
        QueryExecutionContext = {'Database': database},
        ResultConfiguration = {'OutputLocation': output_location, 
                               'EncryptionConfiguration': {'EncryptionOption': encryption_option}})
    
    return query_info

In [8]:
def removeVarCharValue(dct):
    if 'VarCharValue' in dct:
        return dct['VarCharValue']
    if 'Data' in dct:
        d = {}; val = dct['Data']
        for ind, name in zip(range(5), ['SubjID', 'Timestamp', 'X', 'Y', 'Z']):        
            d[name] = val[ind]
        return d 
    return dct

In [9]:
def get_query_results(query_info):
    query_id = query_info['QueryExecutionId']
    
    query_done = False
    
    while not query_done:
        query_exec = client.get_query_execution(QueryExecutionId = query_id)
        status = query_exec['QueryExecution']['Status']['State']
        if status == 'SUCCEEDED':
            query_done = True
        if (status == 'FAILED' or status == 'CANCELLED'):
            print('QUERY FAILED')
            return
        time.sleep(1)

    page_iter = paginator.paginate(QueryExecutionId = query_id)

    for page in page_iter:
        response = page['ResultSet']['Rows']
        break
        
    return len(response) > 1

In [10]:
def isolate_full_queries(QID, QueryPaths, NonEmpty):

    fullQID = []
    fullQueryPaths = []

    for ID, p, full in zip(QID, QueryPaths, NonEmpty):

        if full == True:
            fullQID.append(ID)
            fullQueryPaths.append(p)

    return fullQID, fullQueryPaths

In [11]:
def getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar):
    
    # Initialize Empty Lists for Each Necessary Piece of Information from Each Record
    StartTimestamps = []
    EndTimestamps = []
    SaveFilePaths = []
    SubjID = []

    # Symptom Reports
    Subj = list(timepoints_symt.SubjID)
    StartTimes = timepoints_symt.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_symt.apply(lambda row: row.Timestamp, axis = 1)
    SavePaths = timepoints_symt.apply(lambda row: os.path.join(dest, dest_ext_symt, str(row.SubjID), 
                                                                                   str(row.Timestamp)[:7], 
                                                                                   str(row.Timestamp)[8:10],
                                                                                   str(row.Timestamp)[11:13] +
                                                                                   str(row.Timestamp)[14:16] +
                                                                                   str(row.Timestamp)[17:] + '.csv'),axis = 1)
    SubjID = SubjID + Subj
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    
    # Diaries
    Subj = list(timepoints_diar.SubjID)
    StartTimes = timepoints_diar.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_diar.apply(lambda row: row.Timestamp, axis = 1)
    SavePaths = timepoints_diar.apply(lambda row: os.path.join(dest, dest_ext_diar, str(row.SubjID), 
                                                                                   str(row.Timestamp)[:7], 
                                                                                   str(row.Timestamp)[8:10],
                                                                                   str(row.Timestamp)[11:13] +
                                                                                   str(row.Timestamp)[14:16] +
                                                                                   str(row.Timestamp)[17:] + '.csv'),axis = 1)
    SubjID = SubjID + Subj
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    

    # Medication Reports
    Subj = list(timepoints_med.SubjID)
    StartTimes = timepoints_med.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = -30), axis = 1)
    EndTimes = timepoints_med.apply(lambda row: row.Timestamp + pd.Timedelta(unit = 'minute', value = 30), axis = 1)
    SavePaths = timepoints_med.apply(lambda row: os.path.join(dest, dest_ext_med, str(row.SubjID), 
                                                                                  str(row.Timestamp)[:7], 
                                                                                  str(row.Timestamp)[8:10],
                                                                                  str(row.Timestamp)[11:13] +
                                                                                  str(row.Timestamp)[14:16] +
                                                                                  str(row.Timestamp)[17:] + '.csv'), axis = 1)
    SubjID = SubjID + Subj
    StartTimestamps = StartTimestamps + list(StartTimes)
    EndTimestamps = EndTimestamps + list(EndTimes)
    SaveFilePaths = SaveFilePaths + list(SavePaths)
    
    Records = pd.DataFrame(columns = ['SubjID', 'StartTimestamps', 'EndTimestamps', 'SaveFilePaths'])
    
    Records['SubjID'] = SubjID
    Records['StartTimestamps'] = StartTimestamps
    Records['EndTimestamps'] = EndTimestamps
    Records['SaveFilePaths'] = SaveFilePaths
    
    return Records

In [12]:
### GROUP BY ONE DAY OF DATA
def iterate_reports(timepoints_med, timepoints_symt, timepoints_diar):
    
    Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
    
    Records['StartDate'] = Records['StartTimestamps'].apply(lambda x: x.date())
    
    RecordQueries = []
    QueryPaths = []
    
    # there is a new query (and also destination file paths) required for each patient report
    for subject in Records['SubjID'].unique():
        
        SubjRecords = Records[Records['SubjID'] == subject]
        
        S6 = str(user_id_4_to_6(subject))
        
        for date in SubjRecords['StartDate'].unique():
            
            DateSubjRecords = SubjRecords[SubjRecords['StartDate'] == date]
            DateTimestamps = []
            
            for record in DateSubjRecords.iterrows():
                
                DateTimestamps.append((record[1]['StartTimestamps'], record[1]['EndTimestamps']))
                
            query = generate_query(DateTimestamps, S6, subject)
            RecordQueries.append(query)
            
            ### MAKE FILE NAMES AND SAVE THEM - each query has a file name that goes with it to match up
            
    for query in RecordQueries:
        QueryPaths.append(os.path.join(dest, 
                                       'Queries', 
                                       str(int(query[142:148]) - int(query[15:22])), 
                                       str(query[179:186]), 
                                       str(int(query[142:148]) - int(query[15:22])) + ' ' + str(query[179:189]) + '.csv'))
            
    return RecordQueries, QueryPaths

In [13]:
### GROUP BY TWO DAYS OF DATA
def iterate_reports(timepoints_med, timepoints_symt, timepoints_diar):
    
    Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
    
    # set even dates back to group by two dates
    Records['StartDate'] = Records['StartTimestamps'].apply(lambda x: x.date())
    Records['OddDate'] = Records['StartDate'].apply(lambda x: pd.Timestamp(x) if x.day % 2 == 1 
                                                else pd.Timestamp(x) - pd.Timedelta('1 days'))
    
    RecordQueries = []
    QueryPaths = []
    
    # there is a new query (and also destination file paths) required for each patient report
    for subject in Records['SubjID'].unique():
        
        SubjRecords = Records[Records['SubjID'] == subject]
        
        S6 = str(user_id_4_to_6(subject))
        
        for date in SubjRecords['OddDate'].unique():
            
            DateSubjRecords = SubjRecords[SubjRecords['OddDate'] == date]
            DateTimestamps = []
            
            for record in DateSubjRecords.iterrows():
                
                DateTimestamps.append((record[1]['StartTimestamps'], record[1]['EndTimestamps']))
                
            query = generate_query(DateTimestamps, S6, subject)
            RecordQueries.append(query)
            
            ### MAKE FILE NAMES AND SAVE THEM - each query has a file name that goes with it to match up
            
            QueryPaths.append(os.path.join(dest, 
                                           'Queries', 
                                           str(subject), 
                                           str(pd.Timestamp(date).year) + '-' + str(pd.Timestamp(date).month), 
                                           str(subject) + ' ' + str(date)[:10] + ' ' + 
                                           str(pd.Timestamp(date) + pd.Timedelta('1 days'))[:10] + '.csv'))
            
    return RecordQueries, QueryPaths

In [15]:
# ### FIX MISTAKE - ONLY GENERATE QUERIES IF THEY DID NOT RUN FIRST
# ### GROUP BY ONE MONTH OF DATA
# def iterate_reports(timepoints_med, timepoints_symt, timepoints_diar):
    
#     Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
    
#     # set even dates back to group by two dates
#     Records['StartDate'] = Records['StartTimestamps'].apply(lambda x: x.date())
#     Records['StartMonth'] = Records['StartDate'].apply(lambda x: x.month)
    
#     RecordQueries = []
#     QueryPaths = []
    
#     # there is a new query (and also destination file paths) required for each patient report
#     for subject in Records['SubjID'].unique():
        
#         SubjRecords = Records[Records['SubjID'] == subject]
        
#         S6 = str(user_id_4_to_6(subject))
        
#         for month in SubjRecords['StartMonth'].unique():
            
#             MonthSubjRecords = SubjRecords[SubjRecords['StartMonth'] == month]
#             MonthTimestamps = []
#             longTimestamp = False
            
#             for record in MonthSubjRecords.iterrows():
                
#                 MonthTimestamps.append((record[1]['StartTimestamps'], record[1]['EndTimestamps']))
                
#                 st = record[1]['StartTimestamps']
#                 et = record[1]['EndTimestamps']
#                 if len(str(st)) > 19 or len(str(et)) > 19:
#                     longTimestamp = True
            
#             if longTimestamp:
#                 query = generate_query(MonthTimestamps, S6, subject)
#                 RecordQueries.append(query)
            
#             ### MAKE FILE NAMES AND SAVE THEM - each query has a file name that goes with it to match up
            
#     for query in RecordQueries:
#         QueryPaths.append(os.path.join(dest, 
#                                        'Queries', 
#                                        str(int(query[142:148]) - int(query[15:22])), 
#                                        str(query[179:186]) + '.csv'))
            
#     return RecordQueries, QueryPaths

In [14]:
### GROUP BY ONE MONTH OF DATA
def iterate_reports(timepoints_med, timepoints_symt, timepoints_diar):
    
    Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
    
    # set even dates back to group by two dates
    Records['StartDate'] = Records['StartTimestamps'].apply(lambda x: x.date())
    Records['StartMonth'] = Records['StartDate'].apply(lambda x: x.month)
    
    RecordQueries = []
    QueryPaths = []
    
    # there is a new query (and also destination file paths) required for each patient report
    for subject in Records['SubjID'].unique():
        
        SubjRecords = Records[Records['SubjID'] == subject]
        
        S6 = str(user_id_4_to_6(subject))
        
        for month in SubjRecords['StartMonth'].unique():
            
            MonthSubjRecords = SubjRecords[SubjRecords['StartMonth'] == month]
            MonthTimestamps = []
            
            for record in MonthSubjRecords.iterrows():
                
                MonthTimestamps.append((record[1]['StartTimestamps'], record[1]['EndTimestamps']))
                
            query = generate_query(MonthTimestamps, S6, subject)
            RecordQueries.append(query)
            
            ### MAKE FILE NAMES AND SAVE THEM - each query has a file name that goes with it to match up
            
    for query in RecordQueries:
        QueryPaths.append(os.path.join(dest, 
                                       'Queries', 
                                       str(int(query[142:148]) - int(query[15:22])), 
                                       str(query[179:186]) + '.csv'))
            
    return RecordQueries, QueryPaths

In [62]:
# print the query for each report
RecordQueries, QueryPaths = iterate_reports(timepoints_med, timepoints_symt, timepoints_diar)
print(len(RecordQueries))

150


In [188]:
### clear Queries Directory and Refill with newly Generated Paths
try:
    shutil.rmtree('//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Queries')
    os.mkdir('//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Queries')
except(FileNotFoundError):
    os.mkdir('//FS2.smpp.local\\\\RTO\\\\CIS-PD Study\\Patient Record Correlation\\Queries')

# # create directories down to the day
# for file in QueryPaths:
#     # create directories by day
#     try:
#         os.mkdir(file[:86])
#     except(FileExistsError):
#         continue
#     except(FileNotFoundError):
#         # create directories by month
#         try:
#             os.mkdir(file[:83])
#             os.mkdir(file[:86])
#         # create directories by subject
#         except(FileNotFoundError):
#                 os.mkdir(file[:75])
#                 os.mkdir(file[:83])
#                 os.mkdir(file[:86])

# create subject directories
for file in QueryPaths:
    # create directories by day
    try:
        os.mkdir(file[:75])
    except(FileExistsError):
        continue

In [64]:
### call and record queries

s_time = time.time()

QID = []
NonEmpty = []
query_info = []

# set values for testing the code
atOnce = 5
totalRun = len(RecordQueries)

for i in range(atOnce):
    query_info.append(call_query(RecordQueries[i]))

for query in range(len(RecordQueries)):
# for query in range(totalRun):

    QID.append(query_info[query]['QueryExecutionId'])
    NonEmpty.append(get_query_results(query_info[query]))
    
    if query + atOnce < len(RecordQueries):
#     if query + atOnce < totalRun:
        query_info.append(call_query(RecordQueries[query + atOnce]))

# record the total run time, average time per query, and time to call all queries
print(str(int(((time.time() - s_time) / 60) / 60)) + ' hours ' + 
      str(int(((time.time() - s_time) / 60) % 60)) + ' minutes ' + 
      str(int((time.time() - s_time) % 60)) + ' seconds')

avgQueryTime = (time.time() - s_time) / totalRun
print(str(avgQueryTime) + ' seconds per query')

# print(str((len(RecordQueries) * avgQueryTime) / (60 * 60)) + ' hours for all queries')

fullQID, fullQueryPaths = isolate_full_queries(QID, QueryPaths, NonEmpty)

# get the number of the queries that result in files with data
print(str(len(fullQID)) + '/' + str(totalRun) + ' have data')

QUERY FAILED
6 hours 8 minutes 33 seconds
147.42003095785776 seconds per query
129/150 have data


In [210]:
### MOVE DOWNLOADED QUERY FILES TO COMPREHENSIBLE PATHS
for query, path in zip(fullQID, fullQueryPaths):
    shutil.move(os.path.join('C:\\Users\\rpieper\\Downloads', query + '.csv'), path)

In [252]:
### OPEN FULL MONTH FILE
### GENERATE FILES FOR EACH RECORD FROM QUERY MONTH FILES
Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
Records['StartDate'] = Records['StartTimestamps'].apply(lambda x: x.date())
Records['StartMonth'] = Records['StartDate'].apply(lambda x: x.month)

for subject in Records['SubjID'].unique():
        
    SubjRecords = Records[Records['SubjID'] == subject]

    for month in SubjRecords['StartMonth'].unique():

        MonthSubjRecords = SubjRecords[SubjRecords['StartMonth'] == month]
        
        s_time = time.time()
        
        try:
            MonthQuery = pd.read_csv(os.path.join(dest, 'Queries', str(subject), 
                                                  str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7] + '.csv'), 
                                     parse_dates = ['Timestamp'])
        except(FileNotFoundError):
            continue

        for record in MonthSubjRecords.iterrows():
            indices = ((MonthQuery.Timestamp > record[1]['StartTimestamps']) & 
                       (MonthQuery.Timestamp < record[1]['EndTimestamps']))
            if len(indices) >= 1:
                MonthQuery[indices].to_csv(record[1]['SaveFilePaths'], index = False)
        
        print(str(int(((time.time() - s_time) / 60) / 60)) + ' hours ' + 
              str(int(((time.time() - s_time) / 60) % 60)) + ' minutes ' + 
              str(int((time.time() - s_time) % 60)) + ' seconds')
        
        # remove query file after read to allow starting and stopping of processing?

0 hours 13 minutes 10 seconds


KeyboardInterrupt: 

In [22]:
### SELECTIVELY OPEN QUERIED FILE
### GENERATE FILES FOR EACH RECORD FROM QUERY MONTH FILES
Records = getTimestampsPaths(timepoints_med, timepoints_symt, timepoints_diar)
Records['StartDate'] = Records['StartTimestamps'].apply(lambda x: x.date())
Records['StartMonth'] = Records['StartDate'].apply(lambda x: x.month)

for subject in Records['SubjID'].unique():
        
    SubjRecords = Records[Records['SubjID'] == subject]

    for month in SubjRecords['StartMonth'].unique():

        MonthSubjRecords = SubjRecords[SubjRecords['StartMonth'] == month]
        
        try:
            s_time = time.time()
            MonthQuery = pd.read_csv(os.path.join(dest, 'Queries', str(subject), 
                                                  str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7] + '.csv'), 
                                     parse_dates = ['Timestamp'], usecols = ['Timestamp'])
        except(FileNotFoundError):
            print(str(subject) + ' ' + str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7] + ' NO FILE FOUND')
            continue

        for record in MonthSubjRecords.iterrows():
            indices = ((MonthQuery.Timestamp > record[1]['StartTimestamps']) & 
                       (MonthQuery.Timestamp < record[1]['EndTimestamps']))
            
            start_ind = indices.idxmax()
            length = sum(indices)

            if length >= 1:
                pd.read_csv(os.path.join(dest, 'Queries', str(subject), 
                                         str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7] + '.csv'),
                            skiprows = start_ind, nrows = length).to_csv(record[1]['SaveFilePaths'], index = False, 
                                                                         header = ['SubjID', 'Timestamp', 'X', 'Y', 'Z'])

        print(str(subject) + ' ' + str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7])
        
        # move file to archive to prevent repeats
        shutil.move(os.path.join(dest, 'Queries', str(subject), 
                                 str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7] + '.csv'), 
                    os.path.join(dest, 'Archived Queries',
                                 str(subject) + ' ' + str(MonthSubjRecords['StartDate'][MonthSubjRecords.index[0]])[:7] + 
                                 '.csv'))
        
        print(str(int(((time.time() - s_time) / 60) / 60)) + ' hours ' + 
              str(int(((time.time() - s_time) / 60) % 60)) + ' minutes ' + 
              str(int((time.time() - s_time) % 60)) + ' seconds')

1046 2017-09
0 hours 22 minutes 44 seconds
1046 2017-10
0 hours 36 minutes 8 seconds
1046 2017-11 NO FILE FOUND
1046 2017-12
0 hours 5 minutes 17 seconds
1046 2018-01
0 hours 11 minutes 51 seconds
1046 2018-02 NO FILE FOUND
1046 2017-08
0 hours 3 minutes 11 seconds
1041 2017-09
0 hours 19 minutes 54 seconds
1041 2017-10 NO FILE FOUND
1041 2017-11 NO FILE FOUND
1041 2017-12
0 hours 0 minutes 38 seconds
1041 2017-07
0 hours 3 minutes 2 seconds
1041 2017-08
0 hours 21 minutes 23 seconds
1048 2017-09
1 hours 58 minutes 53 seconds
1048 2017-10
1 hours 10 minutes 39 seconds
1048 2017-11
1 hours 11 minutes 12 seconds
1048 2017-12
1 hours 13 minutes 32 seconds
1048 2018-01
0 hours 37 minutes 6 seconds
1048 2018-02 NO FILE FOUND
1048 2017-08
0 hours 1 minutes 20 seconds
1042 2017-09 NO FILE FOUND
1042 2017-10 NO FILE FOUND
1042 2017-11 NO FILE FOUND
1042 2018-01 NO FILE FOUND
1042 2017-07
0 hours 0 minutes 24 seconds
1042 2017-08 NO FILE FOUND
1000 2017-09
0 hours 9 minutes 2 seconds
1000 2017-

1032 2018-01
1 hours 39 minutes 50 seconds
1032 2017-07
0 hours 13 minutes 18 seconds
1032 2017-08
1 hours 19 minutes 53 seconds
1035 2017-09
0 hours 27 minutes 41 seconds
1035 2017-10
0 hours 33 minutes 14 seconds
1035 2017-11
0 hours 6 minutes 58 seconds
1035 2017-12
0 hours 0 minutes 27 seconds
1035 2018-01
0 hours 8 minutes 19 seconds
1035 2017-07
0 hours 14 minutes 4 seconds
1035 2017-08
0 hours 26 minutes 25 seconds
1035 2018-02 NO FILE FOUND
1035 2018-03 NO FILE FOUND
1038 2017-09
0 hours 46 minutes 9 seconds
1038 2017-10
0 hours 54 minutes 42 seconds
1038 2017-11
0 hours 47 minutes 30 seconds
1038 2017-12
0 hours 35 minutes 28 seconds
1038 2018-01
0 hours 32 minutes 1 seconds
1038 2018-02
0 hours 2 minutes 13 seconds
1038 2017-07
0 hours 12 minutes 1 seconds
1038 2017-08
1 hours 19 minutes 52 seconds
1038 2018-03 NO FILE FOUND
1022 2017-09
0 hours 24 minutes 35 seconds
1022 2017-10
0 hours 12 minutes 38 seconds
1022 2017-11
0 hours 3 minutes 10 seconds
1022 2017-12 NO FILE FOUN

In [123]:
### TIMER
s_time = time.time()



print(str(int(((time.time() - s_time) / 60) / 60)) + ' hours ' + 
      str(int(((time.time() - s_time) / 60) % 60)) + ' minutes ' + 
      str(int((time.time() - s_time) % 60)) + ' seconds')

0 hours 0 minutes 1 seconds


,SubjID,StartTimestamps,EndTimestamps,SaveFilePaths,StartDate,OddDate
0,1046,2017-09-11 15:15:47.000,2017-09-11 15:45:47.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-11,2017-09-11
1,1046,2017-09-11 19:02:50.000,2017-09-11 19:32:50.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-11,2017-09-11
2,1046,2017-09-11 20:02:18.000,2017-09-11 20:32:18.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-11,2017-09-11
3,1046,2017-09-11 21:17:49.000,2017-09-11 21:47:49.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-11,2017-09-11
4,1046,2017-09-13 16:24:42.000,2017-09-13 16:54:42.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-13,2017-09-13
5,1046,2017-09-13 23:32:11.000,2017-09-14 00:02:11.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-13,2017-09-13
6,1046,2017-09-15 13:30:51.000,2017-09-15 14:00:51.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-15,2017-09-15
7,1046,2017-09-15 19:36:53.000,2017-09-15 20:06:53.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-15,2017-09-15
8,1046,2017-09-16 13:21:50.000,2017-09-16 13:51:50.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-16,2017-09-15
9,1046,2017-09-17 02:26:35.000,2017-09-17 02:56:35.000,//FS2.smpp.local\\RTO\\CIS-PD Study\Patient Re...,2017-09-17,2017-09-17
